# Dependencies

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Inside terminal run the following:

```
ollama serve &
ollama run llama3
```






In [2]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [3]:
# Load Google Drive because it stores /content/drive/My Drive/ards-cohort-notes/ards-cohort-notes.csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [4]:
import pandas as pd
import time
import random
import csv
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

# Functions to load data, specify LLM prompt, and perform LLM inference

In [5]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.fillna('', inplace=True)
    return df

def select_random_start(num_rows, min_rows=15):
    if num_rows < min_rows:
        raise ValueError(f"The dataset must contain at least {min_rows} rows to process.")
    return random.randint(0, num_rows - min_rows)

def create_prompt_template():
    return PromptTemplate(
        template=(
            "Context: You are a clinician receiving chunks of radiology reports for patients in an ICU. Please do the reviewing as quickly as possible.\n"
            "Task: Determine if the patient suffered from bilateral infiltrates.\n"
            "Instructions: Answer with 'Yes' or 'No'. If there is not enough information, answer 'No'.\n"
            "Discharge Text:\n{radiology_texts}\n\n"
            "Query: Does the chunk of text mention that the patient suffered from bilateral infiltrates? Answer strictly in 'Yes' or 'No'."
        ),
        input_variables=["radiology_texts"]
    )

def chunk_text(text, chunk_size, overlap):
    start = 0
    chunks = []
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

def check_for_bilateral_infiltrates(radiology_texts, llm, prompt_template, chunk_size, chunk_overlap):
    if not radiology_texts:  # Check if radiology_texts is empty or not
        return None, "No radiology text available", 0  # Return None if there is no radiology_texts entry

    chunks = chunk_text(radiology_texts, chunk_size, chunk_overlap)
    results = []
    for chunk in chunks:
        prompt = prompt_template.format(radiology_texts=chunk)
        try:
            response = llm.invoke(prompt)
            results.append(response.strip())
        except Exception as e:
            results.append(f"Error invoking model: {e}")
    bilateral_infiltrates_mentions = [res for res in results if "Yes" in res]
    if bilateral_infiltrates_mentions:
        return "Yes", bilateral_infiltrates_mentions[0], len(radiology_texts) # Return bilateral_infiltrates result, explanation, and length of radiology_texts
    else:
        return "No", results[0] if results else "No sufficient data", len(radiology_texts) # Return bilateral_infiltrates result, explanation, and length of radiology_texts

def process_patients(df, start_index, num_patients, llm, prompt_template, chunk_size, chunk_overlap, output_csv_file, progress_report_file):
    processing_time = []
    with open(output_csv_file, 'a', newline='') as csvfile, open(progress_report_file, 'a') as report_file:
        # Open the CSV file for writing
        fieldnames = ['hadm_id', 'radiology_texts_length', 'bilateral_infiltrates_detected', 'time_taken']
        # Define the column names
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Only write header if the file is empty
        if csvfile.tell() == 0:
            writer.writeheader()  # Write the header row

        for i in range(start_index, start_index + num_patients):
            current_hadm_id = df['hadm_id'].values[i]
            start_time = time.time()
            data = df[df['hadm_id'] == current_hadm_id]
            if data.empty:
                result = f"No data found for hadm_id: {current_hadm_id}"
            else:
                radiology_texts = data['radiology_texts'].values[0] if 'radiology_texts' in data.columns else ''
                bilateral_infiltrates_result, explanation, radiology_texts_length = check_for_bilateral_infiltrates(radiology_texts, llm, prompt_template, chunk_size, chunk_overlap)
                end_time = time.time()
                elapsed_time = end_time - start_time
                minutes, seconds = divmod(elapsed_time, 60)
                processing_time.append(elapsed_time)
                # Write data to CSV file
                writer.writerow({
                    'hadm_id': current_hadm_id,
                    'radiology_texts_length': radiology_texts_length,
                    'bilateral_infiltrates_detected': bilateral_infiltrates_result,
                    'time_taken': round(elapsed_time)
                })
                csvfile.flush()  # Flush the buffer to ensure data is written

                # Write data to progress report file
                report_file.write(f"Patient Number: {i}, HADM ID: {current_hadm_id}, Radiology Text Length: {radiology_texts_length}, BI Detected: {bilateral_infiltrates_result}, Time Taken: {round(elapsed_time)}\n")
                report_file.flush()  # Flush the buffer to ensure data is written

                print(f"Processed Patient Number {i}\n")

# Main (calls all the functions above)

In [6]:
def main(file_path, model_name, chunk_size, chunk_overlap, output_csv_file, progress_report_file, num_patients):
    df = load_data(file_path)
    #start_index = select_random_start(len(df))
    start_index = 0
    prompt_template = create_prompt_template()
    llm = Ollama(model=model_name, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
    process_patients(df, start_index, num_patients, llm, prompt_template, chunk_size, chunk_overlap, output_csv_file, progress_report_file)

if __name__ == "__main__":
    main(
        file_path='/content/drive/My Drive/ards-cohort-notes/ards-cohort-notes.csv',
        model_name="llama3",
        chunk_size=4096,
        chunk_overlap=100,
        output_csv_file='bilateral-infiltrates-2020.csv',  # Change the output file name to a CSV file
        progress_report_file='bilateral-infiltrates-2020.txt',  # Path to the progress report file
        num_patients = 2021
    )

YesYesNoYesYesYesProcessed Patient Number 0

NoNoProcessed Patient Number 1

YesNo.NoNo.YesNoNo.No.No.NoNo.NoNo.YesYesYesYesNo.Processed Patient Number 2

NoYesNoProcessed Patient Number 3

Yes.Yes.No.Yes.Processed Patient Number 4

NoNoNo.NoProcessed Patient Number 5

No.YesNoNoProcessed Patient Number 6

NoNo.No.No.NoProcessed Patient Number 7

YesYesNo.No.YesYesNoYes.YesProcessed Patient Number 8

NoYesYesNoNo.No.YesYes.Processed Patient Number 9

No.NoNoNoProcessed Patient Number 10

No.NoNo.YesYes.YesProcessed Patient Number 11

NoNoNo.No.NoNoNo.Processed Patient Number 12

NoProcessed Patient Number 13

NoNoNo.Processed Patient Number 14

No.No.NoNo.NoNoProcessed Patient Number 15

YesNo.NoNo.Processed Patient Number 16

NoNoNoNo.NoNoNo.NoNoNo.NoNo.YesYesProcessed Patient Number 17

YesYes.Yes.YesNo.YesYesNoNoNo.No.NoProcessed Patient Number 18

NoNo.NoProcessed Patient Number 19

No.NoNoNoNo.No.Processed Patient Number 20

NoNo.YesNoNo.NoNoNoProcessed Patient Number 21

No.No.Ye